# Brain Tumor Segmentation

This model will segment the tumors from MRI Images and output the location of the brain tumor

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split, Dataset
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

In [13]:
# Data Augmentation
image_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

mask_transform = transforms.Compose([
    transforms.Resize((256,256), interpolation=Image.NEAREST),
    transforms.ToTensor()
])

In [14]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_transform = image_transform
        self.mask_transform = mask_transform
        self.images = sorted(os.listdir(image_dir))


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image[idx])
        mask_path = os.path.join(self.mask_dir, self.mask[idx])

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.image_transform:
            image = self.image_transform(image)

        if self.mask_transform:
            mask = self.mask_transform(Image)

        return image, mask

In [16]:
dataset = SegmentationDataset(
    image_dir = "/kaggle/input/brain-tumor-segmentation/images",
    mask_dir = "/kaggle/input/brain-tumor-segmentation/masks",
    image_transform = image_transform,
    mask_transform = mask_transform
)

In [21]:
dataset_size = len(dataset)

train_size = int(0.70 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - (train_size + val_size)

train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    [train_size, val_size, test_size]
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [24]:
#Building block of the Double convolution of U net

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

In [29]:
## Unet

class UNet(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.down1 = DoubleConv(3,64)
        self.pool1 = nn.MaxPool2d(2)
    
        self.down2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
    
        self.down3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
    
        self.down4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
    
        self.bottleneck = DoubleConv(512, 1024)
    
        self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.conv4 = DoubleConv(1024, 512)
    
        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.conv3 = DoubleConv(512, 256)
    
        self.up2 = nn.ConvTranspose2d(256,128, 2, stride=2)
        self.conv2 = DoubleConv(256, 128)
    
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv1 = DoubleConv(128, 64)
    
        self.final = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self,x):
        c1 = self.down1(x)
        p1 = self.pool1(c1)

        c2 = self.down1(p1)
        p2 = self.pool1(c2)

        c3 = self.down1(p2)
        p3 = self.pool1(c3)

        c4 = self.down1(p3)
        p4 = self.pool1(c4)

        bn = self.bottleneck(p4)

        u4 = self.up4(bn)
        u4 = self.torch.cat([u4, c4], dim=1)
        u4 = self.conv4(u4)

        u3 = self.up3(u4)
        u3 = self.torch.cat([u3, c3], dim=1)
        u3 = self.conv4(u4)

        u2 = self.up2(u3)
        u2 = self.torch.cat([u2, c2], dim=1)
        u2 = self.conv4(u2)

        u1 = self.up4(u2)
        u1 = self.torch.cat([u1, c1], dim=1)
        u1 = self.conv4(u1)

        return torch.sigmoid(self.final(u1))

In [30]:
## Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)